In [1]:
%pip install langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


## Import LangChain related modules

In [2]:
import openai
import os

os.environ['OPENAI_API_KEY'] = '*'

os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [3]:
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Define classes

In [4]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [5]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template
        
    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=['message_history', 'recent_message'],
            template = self.bidding_template
        ).format(
            message_history='\n'.join(self.message_history),
            recent_message=self.message_history[-1])
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string
        

## Define participants and debate topic

In [41]:
character_names = ["Employer", "Contractor","Arbitrator"]
statement = "The Contractor was in charge of the design and construction in a construction project holded by the Employer. In proceeding of the project, disputes have arised over design fee payment between the Parties.<Elaborate>. Both party agree that matter will be arbitrated in accordance with the construction contract, the ADR plan and applicable laws. The Arbitrator is designated."
goal = "Through arbitration and mediation, find solutions that acceptable for both Paries."
word_limit = 50

In [42]:
game_description = f"""Here is the statement of fact in the dispute between the Parties: {statement}.
The participants in the arbitration process are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the statement of fact, elaborate on the details of the dispute.")

def generate_character_claim(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(content=
            f"""
            You are the  {character_name} in this dispute. According to the statement of fact: {statement}, please state on your claim in this dispute. State in {word_limit} words or less. 
            The claim should be as specific as possible, normally include what amount of payment that you want other party to pay, and other issues.
            """
            )
    ]
    character_claim = ChatOpenAI(temperature=1.0)(character_specifier_prompt).content
    return character_claim

def generate_character_header(character_name, character_claim):
    return f"""{game_description}
Your role in this arbitration process is {character_name}.
Your claim is as follows: {character_claim}
"""

def generate_character_system_message(character_name, character_header):
    return SystemMessage(content=(
    f"""{character_header}
You will speak as the {character_name} in an arbitration tribunal.
Arbitrator should make the opening and lead the parties to solve their disputes, conclude the agreement at the end, stop only when both party reach the agreement.
Parties should speak following arbitrator's instruction, provide related proof to support your claims, stop only when you reach agreement with other party.
Everthing you said should taget reaching the {goal}.
Do not say thank you over and over again!!!!
Do not hallucinate facts.
Keep your response to {word_limit} words!
Do not add anything else.
    """
    ))

character_claim = [generate_character_claim(character_name) for character_name in character_names]
character_headers = [generate_character_header(character_name, character_claim) for character_name, character_claim in zip(character_names, character_claim)]
character_system_messages = [generate_character_system_message(character_name, character_headers) for character_name, character_headers in zip(character_names, character_headers)]
                                                                                                                                       

In [43]:
for character_name, character_claim, character_header, character_system_message in zip(character_names, character_claim, character_headers, character_system_messages):
    print(f'\n\n{character_name} claim:')
    print(f'\n{character_claim}')
    print(f'\n{character_header}')
    print(f'\n{character_system_message.content}')



Employer claim:

As the Employer in this dispute, I claim that the Contractor has breached the construction contract by failing to perform the design work satisfactorily and attempting to charge an excessive design fee. I request that the Contractor pay a reduced design fee and reimburse any costs incurred as a result of their inadequate work.

Here is the statement of fact in the dispute between the Parties: The Contractor was in charge of the design and construction in a construction project holded by the Employer. In proceeding of the project, disputes have arised over design fee payment between the Parties.<Elaborate>. Both party agree that matter will be arbitrated in accordance with the construction contract, the ADR plan and applicable laws. The Arbitrator is designated..
The participants in the arbitration process are: Employer, Contractor, Arbitrator.
Your role in this arbitration process is Employer.
Your claim is as follows: As the Employer in this dispute, I claim that th

## Output parser for bids

In [44]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return 'Your response should be an integer delimited by angled brackets, like this: <int>.'     
    
bid_parser = BidOutputParser(
    regex=r'<(\d+)>', 
    output_keys=['bid'],
    default_output_key='bid')

## Generate bidding system message

In [45]:
def generate_character_bidding_template(character_header):
    bidding_template = (
    f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """)
    return bidding_template

character_bidding_templates = [generate_character_bidding_template(character_header) for character_header in character_headers]
                                      

In [46]:
for character_name, bidding_template in zip(character_names, character_bidding_templates):
    print(f'{character_name} Bidding Template:')
    print(bidding_template)

Employer Bidding Template:
Here is the statement of fact in the dispute between the Parties: The Contractor was in charge of the design and construction in a construction project holded by the Employer. In proceeding of the project, disputes have arised over design fee payment between the Parties.<Elaborate>. Both party agree that matter will be arbitrated in accordance with the construction contract, the ADR plan and applicable laws. The Arbitrator is designated..
The participants in the arbitration process are: Employer, Contractor, Arbitrator.
Your role in this arbitration process is Employer.
Your claim is as follows: As the Employer in this dispute, I claim that the Contractor has breached the construction contract by failing to perform the design work satisfactorily and attempting to charge an excessive design fee. I request that the Contractor pay a reduced design fee and reimburse any costs incurred as a result of their inadequate work.


```
{message_history}
```

On the scale

## Use an LLM to create an elaborate on debate topic

In [47]:
goal_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(content=
        f"""{game_description}
        
        Please make the dispute resolution goal more specific. 
        Frame the goal as claims to be discussed and solved on the tribunal.
        Please reply with the specified goals in {word_limit} words or less. 
        Speak directly to the arbitration participants: {*character_names,}.
        Do not add anything else."""
        )
]
specified_goal = ChatOpenAI(temperature=1.0)(goal_specifier_prompt).content

print(f"Original goal:\n{goal}\n")
print(f"Detailed goal:\n{specified_goal}\n")

Original goal:
Through arbitration and mediation, find solutions that acceptable for both Paries.

Detailed goal:
The goal of the arbitration is for the Contractor to claim the unpaid design fee and for the Employer to dispute the payment obligation. The Arbitrator will review the evidence and applicable law, and make a decision on the dispute to determine if the Contractor is entitled to the payment and if the Employer is obligated to pay.



## Define the speaker selection function

In [48]:
@tenacity.retry(stop=tenacity.stop_after_attempt(2),
                    wait=tenacity.wait_none(),  # No waiting time between retries
                    retry=tenacity.retry_if_exception_type(ValueError),
                    before_sleep=lambda retry_state: print(f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."),
                    retry_error_callback=lambda retry_state: 0) # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)['bid'])
    return bid

In [49]:
import numpy as np

def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)
        
    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)
    
    print('Bids:')
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f'\t{agent.name} bid: {bid}')
        if i == idx:
            selected_name = agent.name
    print(f'Selected: {selected_name}')
    print('\n')
    return idx

## Main loop

In [50]:
characters = []
for character_name, character_system_message, bidding_template in zip(character_names, character_system_messages, character_bidding_templates):
    characters.append(BiddingDialogueAgent(
        name=character_name,
        system_message=character_system_message,
        model=ChatOpenAI(temperature=0.2),
        bidding_template=bidding_template,
    ))

In [51]:
max_iters = 30
n = 0

simulator = DialogueSimulator(
    agents=characters,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject('Arbitration goal', specified_goal)
print(f"(Arbitration goal): {specified_goal}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Arbitration goal): The goal of the arbitration is for the Contractor to claim the unpaid design fee and for the Employer to dispute the payment obligation. The Arbitrator will review the evidence and applicable law, and make a decision on the dispute to determine if the Contractor is entitled to the payment and if the Employer is obligated to pay.


Bids:
	Employer bid: 1
	Contractor bid: 1
	Arbitrator bid: 1
Selected: Arbitrator


(Arbitrator): Good afternoon, Employer and Contractor. Thank you for participating in this arbitration process. As the Arbitrator, my goal is to help both parties reach a fair and just resolution to this dispute. Contractor, can you please present your claim and provide any evidence to support it?


Bids:
	Employer bid: 1
	Contractor bid: 1
	Arbitrator bid: 1
Selected: Employer


(Employer): As the Employer, I claim that the Contractor has breached the construction contract by failing to perform the design work satisfactorily and attempting to charge an exc

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 222ca5a6e8d41961dbceaafb6bbdeaf3 in your message.).


(Arbitrator): Thank you, Employer and Contractor, for your cooperation and willingness to work towards a resolution. I am glad that we were able to reach a fair and just decision in this matter. I hope that this resolution will allow both parties to move forward and continue their business relationship. This concludes the arbitration process.


Bids:
	Employer bid: 1
	Contractor bid: 1
	Arbitrator bid: 1
Selected: Employer


(Employer): Thank you, Arbitrator, for your leadership and guidance throughout this arbitration process. We appreciate your efforts in helping us reach a resolution that is acceptable for both parties. We hope that we can continue to work together with the Contractor in the future.


Bids:
	Employer bid: 1
	Contractor bid: 1
	Arbitrator bid: 1
Selected: Contractor


(Contractor): Thank you, Arbitrator, for your dedication to this process and for helping us reach a fair and just resolution. We appreciate the opportunity to present our case and are glad that we were 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 2513ff9eefe0c10f659ef4f6a9c807ba in your message.).


Bids:
	Employer bid: 1
	Contractor bid: 1
	Arbitrator bid: 1
Selected: Employer


(Employer): We would like to formally conclude this arbitration process and thank the Arbitrator for their time and effort in helping us reach a resolution. We are glad that we were able to come to an agreement with the Contractor and fulfill our obligation to pay the full design fee. Thank you again for your assistance.


